In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
import re
import requests
from bs4 import BeautifulSoup
import glob


from fake_useragent import UserAgent
ua = UserAgent()

tqdm_pandas(tqdm())

0it [00:00, ?it/s]

In [3]:
import sys
sys.setrecursionlimit(100000000)

In [6]:
def get_soup(url, timeout=5):
    headers  = {'User-Agent':ua.random}
    try:
        response = requests.get(url,headers=headers)
    except:
        print("FAILED "+ url)
        return 0
    attempts = 0
    while(not response.ok):
            #print((url+' failed with code: '+str(response.status_code)))
            if attempts > timeout:
                print(url+' failed with code: '+str(response.status_code))
                return BeautifulSoup('','lxml')
            response = requests.get(url)
            attempts += 1
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    return soup

In [7]:
def get_beer_stats(row):
    soup = row['soup']
    stats = soup.find(id='item_stats').find('dl')
    row['ba_score']    = soup.find(class_='BAscore_big ba-score').get_text()
    row['num_reviews'] = int(stats.find(class_='ba-reviews').get_text().replace(',',''))
    row['num_ratings'] = int(stats.find(class_='ba-ratings').get_text().replace(',',''))
    row['ravg']        = float(stats.find(class_='ba-ravg').get_text().replace(',',''))
    row['pdev']        = float(stats.find(class_='ba-pdev').get_text().replace(',','').replace('%',''))
    row['wants']       = int(stats.find(class_='ba-wants').get_text().replace(',',''))
    row['gots']        = int(stats.find(class_='ba-gots').get_text().replace(',',''))
    row['for_trade']   = int(stats.find_all('dt')[-1].get_text().replace(',',''))

    info_links = soup.find('div',style="float:right;width:70%;").find_all('a')
    row['brewery_name'] = info_links[0].get_text()
    row['brewery_loation'] = info_links[1].get_text()
    try:
        row['brewery_website'] = info_links[3]['href']
    except:
        row['brewery_website'] = ''
    row['beer_style'] = info_links[-1].get_text()
    row['style_url'] = info_links[-1]['href']
    if row['brewery_website'] == row['style_url']:
        row['brewery_website'] = ''
    try:
        row['abv'] = float(re.findall(r'(?<=\(ABV\): )\d+\.\d+',soup.find('div',style="float:right;width:70%;").get_text())[0])
    except:
        row['abv'] = np.nan
    row['availability'] = re.findall(r'(?<=Availability: )[\w\-]*',soup.find('div',style="float:right;width:70%;").get_text())[0]
    return row

In [159]:
beers = pd.read_pickle('beer_soup_229.pkl')

In [160]:
tqdm_pandas(tqdm())
beers = beers.progress_apply(get_beer_stats,axis=1)


0it [00:00, ?it/s]
100%|██████████| 3/3 [00:00<00:00, 21.46it/s]
5it [00:00, 19.82it/s]                       
7it [00:00, 19.30it/s]
9it [00:00, 18.70it/s]
12it [00:00, 19.55it/s]
14it [00:00, 19.19it/s]
17it [00:00, 20.19it/s]
19it [00:00, 19.93it/s]
21it [00:01, 19.35it/s]
23it [00:01, 19.23it/s]
25it [00:01, 18.03it/s]
27it [00:01, 18.53it/s]
30it [00:01, 19.66it/s]
32it [00:01, 18.74it/s]
34it [00:01, 18.33it/s]
36it [00:01, 18.63it/s]
39it [00:02, 19.38it/s]
41it [00:02, 18.92it/s]
43it [00:02, 18.44it/s]
45it [00:02, 18.30it/s]
47it [00:02, 18.25it/s]
50it [00:02, 19.42it/s]
52it [00:02, 18.97it/s]
54it [00:02, 19.17it/s]
57it [00:02, 20.29it/s]
60it [00:03, 18.44it/s]
62it [00:03, 18.53it/s]
65it [00:03, 19.49it/s]
68it [00:03, 20.20it/s]
71it [00:03, 20.86it/s]
74it [00:03, 21.40it/s]
77it [00:03, 21.17it/s]
80it [00:04, 18.57it/s]
82it [00:04, 17.06it/s]
84it [00:04, 17.60it/s]
87it [00:04, 17.90it/s]
89it [00:04, 17.23it/s]
91it [00:04, 17.93it/s]
93it [00:04, 17.85it/s]
95

In [8]:
def get_beer_df_reviews(row):
    ba_url = 'http://www.beeradvocate.com'
    url_suffix = '?sort=topr&start='
    row['reviews'] = get_beer_reviews(row['soup'])
    if row['num_reviews'] > 25:
        if row['num_reviews'] > 100:
            num_reviews = 100
        else:
            num_reviews = row['num_reviews']
        for i in range(1,num_reviews//25):
            url = ba_url + row['url'] + url_suffix + str(i*25)
            soup = get_soup(url)
            reviews = get_beer_reviews(soup)
            row['reviews'] += reviews
    return row

def get_beer_reviews(soup):
    reviews = []
    for rating in soup.find_all(id='rating_fullview_content_2'):
        for span in rating.find_all('span'):
            span.extract()
        review = rating.get_text().strip().encode('utf-8')
        review = review.replace('rDev','')
        reviews.append(str(review))
    return reviews

In [162]:
tqdm_pandas(tqdm())
beers = beers.progress_apply(get_beer_df_reviews,axis = 1)
beers.head()


0it [00:00, ?it/s]
 94%|█████████▍| 17/18 [00:05<00:00,  2.40it/s]
20it [00:07,  2.37it/s]                        
22it [00:08,  1.83it/s]
25it [00:10,  1.75it/s]
27it [00:10,  2.40it/s]
30it [00:11,  3.30it/s]
33it [00:11,  4.47it/s]
35it [00:12,  2.37it/s]
38it [00:13,  3.25it/s]
40it [00:13,  3.31it/s]
42it [00:14,  2.64it/s]
44it [00:15,  2.86it/s]
47it [00:15,  3.91it/s]
49it [00:15,  3.77it/s]
52it [00:16,  5.04it/s]
56it [00:17,  3.72it/s]
59it [00:19,  2.76it/s]
62it [00:20,  3.10it/s]
64it [00:22,  1.63it/s]
65it [00:24,  1.01it/s]
67it [00:26,  1.04it/s]
68it [00:27,  1.18it/s]
71it [00:27,  1.65it/s]
73it [00:27,  2.25it/s]
75it [00:29,  1.69it/s]
76it [00:31,  1.10s/it]
78it [00:33,  1.03s/it]
80it [00:33,  1.36it/s]
83it [00:33,  1.90it/s]
85it [00:36,  1.29it/s]
87it [00:36,  1.79it/s]
90it [00:36,  2.46it/s]
93it [00:37,  2.48it/s]
95it [00:39,  1.90it/s]
96it [00:39,  1.78it/s]
97it [00:41,  1.36it/s]
101it [00:42,  1.52it/s]
102it [00:44,  1.22it/s]
106it [00:44,  1.7

http://www.beeradvocate.com/beer/profile/13947/30771/?sort=topr&start=75 failed with code: 403



178it [01:29,  1.23s/it]
180it [01:31,  1.15s/it]

http://www.beeradvocate.com/beer/profile/18134/13906/?sort=topr&start=50 failed with code: 403



182it [01:33,  1.06s/it]
184it [01:34,  1.08it/s]
185it [01:36,  1.19s/it]
188it [01:38,  1.03s/it]
191it [01:38,  1.37it/s]
194it [01:38,  1.92it/s]
197it [01:38,  2.67it/s]
199it [01:40,  1.89it/s]
201it [01:40,  2.18it/s]
204it [01:40,  3.01it/s]
207it [01:42,  2.48it/s]
209it [01:44,  1.81it/s]
211it [01:44,  2.10it/s]
214it [01:45,  2.88it/s]
218it [01:45,  3.96it/s]
221it [01:45,  5.32it/s]
225it [01:47,  3.75it/s]
228it [01:49,  2.61it/s]
230it [01:51,  1.57it/s]


,name,url,soup,ba_score,num_reviews,num_ratings,ravg,pdev,wants,gots,for_trade,brewery_name,brewery_loation,brewery_website,beer_style,style_url,abv,availability,reviews
0,Imperial Eclipse Stout - High West Rye,/beer/profile/14936/107388/,"<!DOCTYPE html> <html class=""Public NoJs Logge...",92,30,162,4.24,17.92,27,64,17,FiftyFifty Brewing Co.,California,http://www.fiftyfiftybrewing.com,American Double / Imperial Stout,/beer/style/157/,11.9,Fall,[ 2013 version poured into a snifter. Comes ou...
1,River Horse India Pale Ale,/beer/profile/877/138007/,"<!DOCTYPE html> <html class=""Public NoJs Logge...",83,14,90,3.60,31.94,0,32,0,River Horse Brewing Co.,New Jersey,http://www.riverhorse.com,American IPA,/beer/style/116/,5.7,Year-round,"[ No date on the bottle, but purchased from th..."
2,Hibiscus Wit,/beer/profile/24428/66018/,"<!DOCTYPE html> <html class=""Public NoJs Logge...",84,10,57,3.75,16.80,2,7,0,2nd Shift Brewing,Missouri,http://www.2ndshiftbrewing.com,Witbier,/beer/style/48/,5.2,Rotating,[ Gold with pure white headLight fresh stone f...
3,Kozel,/beer/profile/448/5430/,"<!DOCTYPE html> <html class=""Public NoJs Logge...",78,59,134,3.36,24.70,2,21,0,Pivovar Velké Popovice a.s.,Czech Republic,,Czech Pilsener,/beer/style/40/,5.0,Year-round,[ On tap into a dimple beer mugA - Beer is cle...
4,Summer Wheat Ale,/beer/profile/12375/61223/,"<!DOCTYPE html> <html class=""Public NoJs Logge...",84,19,45,3.74,15.24,1,9,0,Mt. Carmel Brewing Company,Ohio,http://www.mtcarmelbrewingcompany.com,American Pale Wheat Ale,/beer/style/93/,4.8,Summer,[ A: The beer is slightly hazy light yellow in...


In [165]:
beers.to_pickle('test.pkl')

In [180]:
for pkl in glob.glob('data/*.pkl'):
    filename = pkl.split('/')[1]


beer_soup_10119.pkl
beer_soup_10349.pkl
beer_soup_10579.pkl
beer_soup_10809.pkl
beer_soup_11039.pkl
beer_soup_11269.pkl
beer_soup_1149.pkl
beer_soup_11499.pkl
beer_soup_11729.pkl
beer_soup_11959.pkl
beer_soup_12189.pkl
beer_soup_12419.pkl
beer_soup_12649.pkl
beer_soup_12879.pkl
beer_soup_13109.pkl
beer_soup_13339.pkl
beer_soup_13569.pkl
beer_soup_1379.pkl
beer_soup_13799.pkl
beer_soup_14029.pkl
beer_soup_14259.pkl
beer_soup_14489.pkl
beer_soup_14719.pkl
beer_soup_14949.pkl
beer_soup_15179.pkl
beer_soup_15409.pkl
beer_soup_15639.pkl
beer_soup_15869.pkl
beer_soup_1609.pkl
beer_soup_16099.pkl
beer_soup_16329.pkl
beer_soup_16559.pkl
beer_soup_16789.pkl
beer_soup_17019.pkl
beer_soup_17249.pkl
beer_soup_17479.pkl
beer_soup_17709.pkl
beer_soup_17939.pkl
beer_soup_18169.pkl
beer_soup_1839.pkl
beer_soup_18399.pkl
beer_soup_18629.pkl
beer_soup_18859.pkl
beer_soup_19089.pkl
beer_soup_19319.pkl
beer_soup_19549.pkl
beer_soup_19779.pkl
beer_soup_20009.pkl
beer_soup_20239.pkl
beer_soup_20469.pkl
beer

In [181]:
tqdm_pandas(tqdm())
for pkl in tqdm(glob.glob('data/*.pkl')):
    temp = pd.read_pickle(pkl)
    temp = temp.apply(get_beer_stats, axis=1)
    temp = temp.apply(get_beer_df_reviews, axis=1)
    filename = pkl.split('/')[1]
    temp.to_pickle('temp/'+filename)






0it [00:00, ?it/s]









  0%|          | 0/89 [00:00<?, ?it/s]





  1%|          | 1/89 [04:50<7:05:45, 290.29s/it]





  2%|▏         | 2/89 [08:44<6:36:36, 273.53s/it]





  3%|▎         | 3/89 [11:50<5:54:11, 247.11s/it]





  4%|▍         | 4/89 [14:18<5:08:03, 217.45s/it]





  6%|▌         | 5/89 [19:27<5:43:01, 245.01s/it]





  7%|▋         | 6/89 [22:48<5:20:25, 231.63s/it]





  8%|▊         | 7/89 [25:45<4:54:25, 215.44s/it]





  9%|▉         | 8/89 [29:49<5:02:19, 223.95s/it]





 10%|█         | 9/89 [32:37<4:36:13, 207.17s/it]





 11%|█         | 10/89 [35:23<4:16:20, 194.68s/it]





 12%|█▏        | 11/89 [38:28<4:09:28, 191.90s/it]





 13%|█▎        | 12/89 [41:23<3:59:51, 186.90s/it]





 15%|█▍        | 13/89 [43:37<3:36:35, 170.99s/it]





 16%|█▌        | 14/89 [46:44<3:39:31, 175.62s/it]





 17%|█▋        | 15/89 [50:08<3:47:25, 184.40s/it]





 18%|█▊        | 16/89 [53:31<3:51:03, 189.91s/it]





 19%|█▉        | 17/89 [57:20<4:01:5

In [6]:
beers = pd.read_pickle(glob.glob('temp/*.pkl')[0])

In [11]:
pkl = 'data/beer_soup_missing.pkl'
temp = pd.read_pickle(pkl)
temp = temp.apply(get_beer_stats, axis=1)
temp = temp.apply(get_beer_df_reviews, axis=1)

In [12]:
dfs = [temp]
for pkl in tqdm(glob.glob('temp/*.pkl')):
    temp = pd.read_pickle(pkl)
    temp.drop('soup',axis='columns',inplace=True)
    dfs.append(temp)
    
beer_reviews = pd.concat(dfs)
dfs = []
beer_reviews.to_pickle('all_beers_reviews.pkl')


100%|██████████| 89/89 [17:23<00:00, 11.20s/it]


NameError: name 'beer_reviews' is not defined

NameError: name 'beer_reviews' is not defined